In [1]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

/aiffel/aiffel/aimmo/mmdetection


In [8]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

0.13.6


In [3]:
# # 완디비 로그인
wandb.login(key= '')

wandb: Currently logged in as: wodms. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [4]:
wandb.init(project="r-cnn", entity="aimmo")

wandb: Currently logged in as: wodms (aimmo). Use `wandb login --relogin` to force relogin


In [9]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_texts')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [ ]:
config_file = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg = Config.fromfile(config_file)

cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'


cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_test.txt'
cfg.data.test.img_prefix = 'test/image'

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg.work_dir = 'middle_fulldata'

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
#cfg.evaluation.save_best='auto'
cfg.data.train.pipeline[2].img_scale=[(960,520)]#['Resize']
cfg.train_pipeline[2].img_scale=[(960,520)]
cfg.test_pipeline[1].img_scale=[(960,520)]
cfg.data.val.pipeline[1].img_scale = [(960,520)]
cfg.data.test.pipeline[1].img_scale = [(960,520)]
cfg.runner.max_epochs = 15

cfg.data.samples_per_gpu = 14 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 0 # Worker to pre-fetch data for each single GPU

In [7]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'r-cnn', 'entity' : 'aimmo', 'name' : 'fulldata'},)] 

In [9]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-07 13:39:53,343 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-07 13:39:53,487 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-12-07 13:39:53,647 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([16]).
20

2022-12-07 13:45:10,229 - mmdet - INFO - Epoch [1][100/1494]	lr: 3.976e-03, eta: 19:11:57, time: 3.098, data_time: 1.548, memory: 11277, loss_rpn_cls: 0.0396, loss_rpn_bbox: 0.0434, loss_cls: 0.5231, acc: 81.9155, loss_bbox: 0.5080, loss: 1.1141
2022-12-07 13:50:22,942 - mmdet - INFO - Epoch [1][200/1494]	lr: 7.972e-03, eta: 19:12:10, time: 3.127, data_time: 1.563, memory: 11277, loss_rpn_cls: 0.0296, loss_rpn_bbox: 0.0403, loss_cls: 0.2358, acc: 91.8817, loss_bbox: 0.3190, loss: 0.6247
2022-12-07 13:55:38,012 - mmdet - INFO - Epoch [1][300/1494]	lr: 1.197e-02, eta: 19:11:39, time: 3.151, data_time: 1.589, memory: 11277, loss_rpn_cls: 0.0282, loss_rpn_bbox: 0.0414, loss_cls: 0.2196, acc: 92.3308, loss_bbox: 0.3022, loss: 0.5915
2022-12-07 14:00:54,904 - mmdet - INFO - Epoch [1][400/1494]	lr: 1.596e-02, eta: 19:10:28, time: 3.169, data_time: 1.603, memory: 11277, loss_rpn_cls: 0.0265, loss_rpn_bbox: 0.0415, loss_cls: 0.2143, acc: 92.5000, loss_bbox: 0.2948, loss: 0.5771
2022-12-07 14:06

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.1 task/s, elapsed: 454s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-07 15:05:53,293 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 33812 | 0.956  | 0.913 |
| truck      | 3084  | 7727  | 0.854  | 0.748 |
| bus        | 1786  | 5248  | 0.898  | 0.818 |
| pedestrian | 3063  | 11764 | 0.796  | 0.625 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.776 |
+------------+-------+-------+--------+-------+
2022-12-07 15:05:53,393 - mmdet - INFO - Epoch(val) [1][2323]	AP50: 0.7760, mAP: 0.7757
2022-12-07 15:11:10,156 - mmdet - INFO - Epoch [2][100/1494]	lr: 2.000e-02, eta: 17:06:44, time: 3.162, data_time: 1.639, memory: 11277, loss_rpn_cls: 0.0188, loss_rpn_bbox: 0.0350, loss_cls: 0.1650, acc: 94.0186, loss_bbox: 0.2488, loss: 0.4676
2022-12-07 15:16:27,080 - mmdet - INFO - Epoch [2][200/1494]	lr: 2.000e-02, eta: 17:06:04, time: 3.169, data_time: 1.609, memory: 11277, loss_rp

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.1 task/s, elapsed: 455s, ETA:     0s
---------------iou_thr: 0.5---------------


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.1 task/s, elapsed: 454s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-08 09:57:03,633 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 19177 | 0.960  | 0.945 |
| truck      | 3084  | 6207  | 0.921  | 0.879 |
| bus        | 1786  | 3093  | 0.936  | 0.907 |
| pedestrian | 3063  | 6527  | 0.854  | 0.777 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.877 |
+------------+-------+-------+--------+-------+
2022-12-08 09:57:03,766 - mmdet - INFO - Epoch(val) [14][2323]	AP50: 0.8770, mAP: 0.8769
2022-12-08 10:02:22,239 - mmdet - INFO - Epoch [15][100/1494]	lr: 2.000e-04, eta: 1:09:18, time: 3.179, data_time: 1.597, memory: 11277, loss_rpn_cls: 0.0047, loss_rpn_bbox: 0.0206, loss_cls: 0.0816, acc: 96.8391, loss_bbox: 0.1515, loss: 0.2584
2022-12-08 10:07:40,386 - mmdet - INFO - Epoch [15][200/1494]	lr: 2.000e-04, eta: 1:04:21, time: 3.181, data_time: 1.601, memory: 11277, loss_r

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 462s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-08 11:24:19,346 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 19192 | 0.961  | 0.945 |
| truck      | 3084  | 6096  | 0.920  | 0.879 |
| bus        | 1786  | 3066  | 0.934  | 0.906 |
| pedestrian | 3063  | 6490  | 0.854  | 0.777 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.877 |
+------------+-------+-------+--------+-------+
2022-12-08 11:24:19,472 - mmdet - INFO - Epoch(val) [15][2323]	AP50: 0.8770, mAP: 0.8767


learning_rate,▄█████████████████████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▂▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█████████████
train/loss,█▇▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,█▇▆▅▅▅▄▄▄▄▄▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_cls,█▇▅▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,██▇▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▄▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_cls,█▇▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/AP50,▁▃▅▅▆▇▇▇███████
val/mAP,▁▃▅▅▆▇▇▇███████
learning_rate,0.0002
